# New quantum walk model

## Loading modules

In [ ]:
using QuantumWalk
using LightGraphs
using SparseArrays
import QuantumWalk: evolve, measure, check_qwdynamics, QWEvolution

## Stochastic Walk

We will define classical stochastic walk evolution. 

In [ ]:
abstract type AbstractStochastic <: QWModelDiscr end

struct Stochastic{T<:AbstractMatrix{<:Real}, G<:SimpleGraph} <: AbstractStochastic
  stochastic_matrix::T
  graph::G
end

function stochastic_matrix(model::AbstractStochastic)
  return model.stochastic_matrix
end

This is a basic description. Let us define functions needed for `QWEvolution`, i.e. `evolve`, `check_qwdynamics` and `measure`.

In [ ]:
function QWEvolution(stoch::AbstractStochastic)
   parameters = Dict{Symbol,Any}(:stochastic => stochastic_matrix(stoch))
   QWEvolution(stoch, parameters)
end

function check_qwdynamics(::Type{QWEvolution},
                          abs_stoch::Stochastic,
                          parameters::Dict{Symbol,Any})
  @assert :stochastic ∈ keys(parameters) "parameters needs to have key stochastic"
  n = nv(graph(abs_stoch))
  @assert isa(parameters[:stochastic], AbstractMatrix{<:Real}) "value for :stochastic needs to be sparse matrix with real numbers"
  @assert size(parameters[:stochastic], 1) == size(parameters[:stochastic], 2) "Stochastic matrix needs to be square stochastic matrix"
  @assert mapslices(sum, parameters[:stochastic], 1)[1,:] ≈ ones(n) "Stochastic matrix needs to be square stochastic matrix of order graph"
end

function stochastic_evolution(s::AbstractMatrix{T}, v::AbstractVector{T}) where T<:Real
  s*v
end

function evolve(qws::QWDynamics{<:AbstractStochastic}, state)
  stochastic_evolution(parameters(qws)[:stochastic], state)
end

function measure(::QWDynamics{<:AbstractStochastic}, state::AbstractVector{<:Real})
   return state
end

function measure(::QWDynamics{<:AbstractStochastic},
                 state::AbstractVector{<:Real},
                 vertices::Vector{Int})
   return state[vertices]
end

Note the functions are defined mostly for `AbstractStochastic`. The definition above describes arbitrary random walk. Let us utilize the method for determining final state.

In [ ]:
n = 11
a = adjacency_matrix(CycleGraph(n))/2
qwe = QWEvolution(Stochastic(a, CycleGraph(n)));

init_state = spzeros(Float64, n)
init_state[1] = 1
println("10 steps")
println(execute(qwe, init_state, 10))
println("100 steps")
println(execute(qwe, init_state, 100))
println("1000 steps")
println(execute(qwe, init_state, 1000))

We see, that the probability converge to the uniform state as expected.

Note there is a very special case of random walk, where each neighbouring node is chosen with equal probability. We can easily define such scenario, with the functions defined above, as they were defined for `AbstractStochastic`.

In [ ]:
struct UniformStochastic{G<:SimpleGraph} <: AbstractStochastic
  graph::G
end

function stochastic_matrix(model::UniformStochastic)
  return QuantumWalk.default_stochastic(graph(model))
end

n = 11
qwe = QWEvolution(UniformStochastic(CycleGraph(n)));

init_state = spzeros(Float64, n)
init_state[1] = 1
println("10 steps")
println(execute(qwe, init_state, 10))
println("100 steps")
println(execute(qwe, init_state, 100))
println("1000 steps")
println(execute(qwe, init_state, 1000))

Note we can analogically define `QWSearch` for the model above.